System found in **Pymablock: an algorithm and a package for
quasi-degenerate perturbation theory**: arXiv:2404.03728v1

In [1]:
# Objects for defining the Hamiltonian
from SymPT import RDSymbol, RDBasis, BosonOp, Dagger, Block, Operator
# Objects for obtaining the Effective Hamiltonian
from SymPT import EffectiveFrame

# Extra utils
from SymPT import display_dict, group_by_operators

# Import sympy
import sympy as sp

# Setup

In [2]:
# ---------------- Defining the symbols ------------------
# Order 0
omega_t = RDSymbol('omega_t', order=0, positive=True, real=True)
omega_r = RDSymbol('omega_r', order=0, positive=True, real=True)
alpha   = RDSymbol('alpha', order=0, positive=True, real=True)
# Order 1
g = RDSymbol('g', order=1, positive=True, real=True)

# ----------------- Defining the basis -------------------
# Boson basis transmon: Infinite bosonic Hilbert space
a_t  = BosonOp('a_t')
ad_t = Dagger(a_t)
# Boson basis resonator: Infinite bosonic Hilbert space
a_r  = BosonOp('a_r')
ad_r = Dagger(a_r)

# -------------- Defining the Hamiltonian ----------------
# Unperturbed Hamiltonian H0
H0 = -omega_t * ad_t * a_t + omega_r * ad_r * a_r + sp.Rational(1,2) * alpha * ad_t * ad_t * a_t * a_t
display(H0)
# Interaction Hamiltonian V
V = -g * (ad_t - a_t) * (ad_r - a_r)
display(V)

alpha*Dagger(a_t)**2*a_t**2/2 + omega_r*Dagger(a_r)*a_r - omega_t*Dagger(a_t)*a_t

-g*(Dagger(a_t) - a_t)*(Dagger(a_r) - a_r)

In [3]:
# Deffining Effective Hamiltonian Object
Eff_frame = EffectiveFrame(H0, V)

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│  a_r   │ Bosonic │      ∞      │
├────────┼─────────┼─────────────┤
│  a_t   │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




# Standard Schrieffer-Wolff Transformation

In [4]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=False, mask=None)
# Obtaining the result in the dict_operator form
H_eff_SWT = Eff_frame.get_H(return_form='operator')
# display_dict(H_eff_SWT)

Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  2.34it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Projecting to operator form: 100%|██████████| 20/20 [00:00<00:00, 392.46it/s]


In [5]:
# substitution dictionary for transmon ground state
subs_nt0 = {
    a_t : 0,
    ad_t : 0
}
subs_nt1 = {
    ad_t*a_t : 1,
}

# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_frame.corrections.values()]

In [6]:
display_dict(group_by_operators(H0))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
display(group_by_operators(sp.factor_terms(H2.expand()))[a_r**2].subs(ad_t*a_t, sp.symbols("N_t")).factor())
display(group_by_operators(sp.factor_terms(H2.expand()))[ad_r**2].subs(ad_t*a_t, sp.symbols("N_t")).factor())

g**2*(2*alpha*N_t - omega_t)/((alpha*N_t - omega_r - omega_t)*(alpha*N_t + omega_r - omega_t))

g**2*(2*alpha*N_t - omega_t)/((alpha*N_t - omega_r - omega_t)*(alpha*N_t + omega_r - omega_t))

In [8]:
#group_by_operators(sp.factor_terms(H2.expand()))[ad_r*a_r*ad_t*a_t].expand().subs(ad_t*a_t, sp.symbols("N_t")).simplify().factor()

In [9]:
group_by_operators(sp.factor_terms(H2.subs(subs_nt1).subs(subs_nt0).expand()))[1].simplify()

g**2*(-alpha - 3*omega_r - omega_t)/(alpha*omega_r + alpha*omega_t + omega_r**2 - omega_t**2)

In [10]:
group_by_operators(sp.factor_terms(H2.subs(subs).expand()))[ad_r*a_r].simplify()

NameError: name 'subs' is not defined

In [ ]:
group_by_operators(sp.factor_terms(H2.subs(subs).expand()))[ad_r**2].simplify()

In [ ]:
group_by_operators(sp.factor_terms(H2.subs(subs).expand()))[a_r**2].simplify()

# Multiblock

In [12]:
# Deffining the mask
mask = Block(inf=a_r*a_t) +  Block(inf=ad_r*a_t) + Block(inf=a_r**2) + Block(inf=a_t**2) + Block(inf=a_t**2*a_r**2)
# Calculate the effective model using the Mask routine up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=False, mask=mask)
H_eff_Mask = Eff_frame.get_H(return_form='operator')
# display_dict(H_eff_Mask)

[-1, -1]
[1, -1]
[-1, -1]
[1, -1]
[-2]
[-1, -1]
[1, -1]
[-2]
[-2]
[-1, -1]
[1, -1]
[-2]
[-2]
[-2, -2]
[-1, -1]
[1, -1]
[-2]
[-2]
[-2, -2]
The perturbative interaction will be added to the full Hamiltonian
[1, 1]
[1, -1]
[2]
[2]
[2, 2]
[1, 1]
[1, -1]
[2]
[2]
[2, 2]
[-1, -1]
[1, -1]
[-2]
[-2]
[-2, -2]
[1, 1]
[1, -1]
[2]
[2]
[2, 2]


Solving for each order: 100%|██████████| 2/2 [00:01<00:00,  1.24it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Projecting to operator form: 100%|██████████| 8/8 [00:00<00:00, 443.41it/s]


In [11]:
Block(inf=ad_r*a_t)

[1, -1]


In [14]:
# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_frame.corrections.values()]

In [15]:
display_dict(group_by_operators(H0))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
display_dict(group_by_operators(sp.factor_terms(H2.expand())))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
group_by_operators(sp.factor_terms(H2.expand())).keys()

dict_keys([Dagger(a_r)*a_r*Dagger(a_t)*a_t, Dagger(a_r)*a_r*(Dagger(a_t)*a_t)**2, Dagger(a_t)*a_t, (Dagger(a_t)*a_t)**2, 1, Dagger(a_r)*a_r])

In [ ]:
group_by_operators(sp.factor_terms(H2.expand()))[(ad_t*a_t)**2].subs(ad_t*a_t, sp.symbols("N_t")).simplify().factor()

alpha*g**2/((alpha*N_t + omega_r - omega_t)*(alpha*N_t - alpha + omega_r - omega_t))

# Full-diagonalization

In [ ]:
# Calculate the effective model using the Full Diagonalization routine up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=True, mask=None)
H_eff_FD = Eff_frame.get_H(return_form='dict_operator')
# display_dict(H_eff_FD)